In [ ]:
from googleAPI import *
from pymongo import MongoClient
from crawling import ProgressBar
from pprint import pprint


In [ ]:
client = MongoClient(
    "mongodb+srv://cluster0.c6ccx.mongodb.net",
    authsource="$external",
    authmechanism="MONGODB-X509",
    tls=True,
    tlsCertificateKeyFile="../../../mongodb.pem",
)
db = client["youtube"]
rawVideoSearch = db["rawVideoSearch"]
videoSearch = db["videoSearch"]
videoCategories = db["videoCategories"]
videoFavorite = db["videoFavorite"]


In [ ]:
with open("apikey.txt", encoding="utf8") as f:
    apikey1 = f.readline().strip()
    apikey2 = f.readline().strip()


In [ ]:
global_query = {"key": apikey2, "maxResults": 100, "regionCode": "KR"}


In [ ]:
import datetime

# 1970-01-01T00:00:00Z
def strftime(datetime: datetime.datetime):
    return datetime.strftime("%Y-%m-%dT%H:%M:%SZ")


def strptime(str):
    return datetime.datetime().strptime(str, "%Y-%m-%dT%H:%M:%SZ")


# search with keyword(costs to much(100 times much than other))
def get_searched_video(query):
    init_query = {**global_query, "part": "id, snippet", "type": "video", "safeSearch": "strict"}
    for key in query:
        init_query[key] = query[key]
    return get_youtube_api_json("search", init_query)


In [ ]:
# get top view count video of korea from search
rawVideoSearch = db["rawVideoSearch"]
start_date = strftime(datetime.datetime(2019, 1, 1))
end_date = strftime(datetime.datetime(2020, 1, 1))
init_query = {"publishedAfter": start_date, "publishedBefore": end_date}
total_search_data = []
nextPageToken = ""
for i in range(20):
    init_query["pageToken"] = nextPageToken
    search_data = get_searched_video(init_query)
    total_search_data.append(search_data)
    nextPageToken = search_data.get("nextPageToken")
    if nextPageToken is None:
        print(search_data)
        break
rawVideoSearch.insert_one({"searchDate": datetime.datetime.now(),"publishedAfter": start_date, "publishedBefore": end_date, "result":total_search_data})


In [ ]:
videoSearch.find()


In [ ]:
# get category data of korea
videoCategories = db["videoCategories"]
category_data = get_youtube_api_json("videoCategories", {"key": apikey1, "part": "snippet, id", "regionCode": "KR"})[
    "items"
]
videoCategories.insert_many(category_data)


In [ ]:
list(videoCategories.find())


In [ ]:
def get_populer_video(query):
    init_query = {**global_query,
        "part": "snippet, contentDetails, status, statistics, player, topicDetails, recordingDetails",
        "chart": "mostPopular",
    }
    for key in query:
        init_query[key] = query[key]
    return get_youtube_api_json("videos", init_query)


def get_video(query):
    init_query = {**global_query,
        "part": "snippet, contentDetails, status, statistics, player, topicDetails, recordingDetails"
    }
    for key in query:
        init_query[key] = query[key]
    return get_youtube_api_json("videos", init_query)


In [ ]:
# videoSearch = db["videoSearch"]
# rawVideoSearch = db["rawVideoSearch"]
# rawVideoSearch.insert_many(videoSearch.find())
# videoSearch.drop()


In [ ]:
import re

hangul = re.compile("[ㄱ-ㅎ가-힣]+")
rawVideoSearch = db["rawVideoSearch"]
videoSearch = db["videoSearch"]
total_pre_data = []
for datas in rawVideoSearch.find():
    for data in datas["items"]:
        try:
            preData = get_video({"id": data["id"]["videoId"]})["items"][0]
        except:
            continue
        if len(hangul.findall(preData["snippet"]["title"])) > 0:
            print(preData["snippet"]["title"])
            total_pre_data.append(preData)
videoSearch.insert_one({"searchDate": datetime.datetime.now(), "info": total_pre_data})


In [ ]:
data = list(videoSearch.find())
pre_data = []
for i in data:
    for j in i["info"]:
        pre_data.append((int(j["statistics"]["viewCount"]), j["snippet"]["title"].strip()))
pre_data.sort(reverse=True)
pre_data


In [ ]:
for category_data in videoCategories.find():
    categoryId = category_data["id"]
    print(categoryId)


In [ ]:
get_video({"id": "8Syy24YLOQw,Iim-a-dmND8,Ws57hamuOZk"})


In [ ]:
start_date = strftime(datetime.datetime(2019, 1, 1))
end_date = strftime(datetime.datetime(2020, 1, 1))
rawVideoSearch = db["rawVideoSearch"]
videoSearch = db["videoSearch"]
total_data = []
for datas in rawVideoSearch.find_one({"publishedAfter": start_date, "publishedBefore": end_date})["result"]:
    data_id = []
    for data in datas["items"]:
        try:
            data_id.append(data["id"]["videoId"])
        except:
            continue
    total_data.append(get_video({"id":",".join(data_id)}))
videoSearch.insert_one({"searchDate": datetime.datetime.now(),"publishedAfter": start_date, "publishedBefore": end_date, "result":total_data})

In [ ]:
videoFavorite = db["videoFavorite"]
total_data = []
for category_data in videoCategories.find():
    categoryId = category_data["id"]
    nextPageToken = ""
    for i in range(20):
        favorite_data = get_populer_video({"videoCategoryId": categoryId, "pageToken": nextPageToken})
        nextPageToken = favorite_data.get("nextPageToken")
        total_data.append({"searchDate": datetime.datetime.now(), "searchCategory": categoryId, "info": favorite_data})
        if nextPageToken is None:
            break
    break
print(total_data)
# videoFavorite.insert_many(totalData)


In [ ]:
init_data = get_populer_video()
total_results = init_data["pageInfo"]["totalResults"]
print(total_results)
# nextPageToken = init_data.get("nextPageToken")
# populer_video_data = [init_data]
# for i in range(5):
#     print(nextPageToken)
#     data = get_populer_video(nextPageToken)
#     populer_video_data.append(data)
#     nextPageToken = data.get("nextPageToken")
#     if nextPageToken is None:
#         break
# pprint(populer_video_data)


In [ ]:
get_youtube_api_json(
    "commentThreads",
    {**global_query,
        "part": "id, replies, snippet",
        "textFormat": "plainText",
        "videoId": "zWhqy34zvVI",
        "order": "relevance",
        # "searchTerms":"차준환"
    },
)
